In [1]:
import pandas as pd
import os

import COMBINE_harmonizer
from COMBINE_harmonizer import cfg

## 01. init

In [2]:
study_name = COMBINE_harmonizer.STUDY_OC
sheet_name = COMBINE_harmonizer.SHEET_FOLLOW_UP

root_dir = '..'

In [3]:
COMBINE_harmonizer.init(f'{root_dir}/config.yaml')

## 02. Dictionary

In [4]:
data_dict_filename = f'{root_dir}/Dictionary_HIE_clinical_variables.xlsx'

In [5]:
df_dict = COMBINE_harmonizer.load_data_dict(data_dict_filename, sheet_name)

In [6]:
dict_columns = list(df_dict[study_name].unique())

## 03. OC

In [7]:
the_dir = cfg.config[f'{study_name}_follow_up_dir']

In [8]:
filenames = os.listdir(the_dir)
filenames.sort()

In [9]:
dfs = [pd.read_csv(os.sep.join([the_dir, each_filename])) for each_filename in filenames]

### 03.1. check columns in each file

In [10]:
for idx, each_df in enumerate(dfs):
    print(f'({idx}/{len(dfs)}) filename: {filenames[idx]} columns: {list(each_df.columns)}')

(0/10) filename: of01.csv columns: ['LHFOLNUM', 'SITENM', 'LAST', 'FIRST', 'HTHRM_ID', 'PROTID02', 'PROTID03', 'PROTID04', 'PROTID05', 'PROTID06', 'PROTID07', 'PROTID08', 'PROTID09', 'PROTID10', 'PROTID11', 'PROTID12', 'PROTID13', 'PROTID14', 'PROTID15', 'PROTID16', 'PROTID17', 'PROTID18', 'PROTID19', 'PROTID20', 'REC_CMP', 'CENTER', 'BIRTHDT', 'VISITDT', 'BIRTHNM', 'PREVCNTR', 'CMP_DATE', 'CRT_DATE']
(1/10) filename: of03.csv columns: ['LHFOLNUM', 'CFSUPER', 'NF3MONEY', 'CFWKPCAR', 'CFWKOCAR', 'CFSCCARE', 'CFSCADUL', 'CFOTHLAN', 'CFSECLAN', 'CFOTHSEC', 'CFNDNRSV', 'CFNDNRSH', 'CFNDOTPT', 'CFNDSPCH', 'CFNDEALY', 'CFNDSOCL', 'CFNDPUL', 'CFNDOPH', 'CFNDGAS', 'CFNDAUD', 'CFNDNEU', 'CFNDOTH', 'CFCVOSP', 'CFNDEVAL', 'CFREGDR', 'CFCCFAC', 'CFCAROT', 'CFTRDYCR', 'CFSPDYCR', 'CFHMDYCR', 'CFBBYSIT', 'CFINITS', 'REC_CMP', 'CENTER', 'CFVISDT', 'CFBRTHDT', 'CFCHRAGE', 'CFADJAGE', 'CFCARE', 'CFOCAR', 'CFMARITL', 'CFLIVING', 'CFPEOPLE', 'CFGRPCAR', 'CFGROCAR', 'CFINSUR', 'CFLANG', 'CFSECOND', 'CFNMP

### 03.2 all columns

In [11]:
all_columns = []
for idx, each_df in enumerate(dfs):
    all_columns += list(map(lambda x: {'column': x, 'idx': idx, 'filename': filenames[idx]}, list(each_df.columns)))
df_columns = pd.DataFrame(all_columns)
df_columns['column:filename'] = df_columns.apply(lambda x: f"{x['column']}:{x['filename']}", axis=1)
df_columns['column:filename:idx'] = df_columns.apply(lambda x: f"{x['column']}:{x['filename']}:{x['idx']}", axis=1)
df_columns.sort_values(by=['filename', 'column'], inplace=True)

In [12]:
df_columns

,column,idx,filename,column:filename,column:filename:idx
26,BIRTHDT,0,of01.csv,BIRTHDT:of01.csv,BIRTHDT:of01.csv:0
28,BIRTHNM,0,of01.csv,BIRTHNM:of01.csv,BIRTHNM:of01.csv:0
25,CENTER,0,of01.csv,CENTER:of01.csv,CENTER:of01.csv:0
30,CMP_DATE,0,of01.csv,CMP_DATE:of01.csv,CMP_DATE:of01.csv:0
31,CRT_DATE,0,of01.csv,CRT_DATE:of01.csv,CRT_DATE:of01.csv:0
...,...,...,...,...,...
401,NF12NRDS,9,of12.csv,NF12NRDS:of12.csv,NF12NRDS:of12.csv:9
400,NF12NRPB,9,of12.csv,NF12NRPB:of12.csv,NF12NRPB:of12.csv:9
434,OF12CHGM,9,of12.csv,OF12CHGM:of12.csv,OF12CHGM:of12.csv:9
420,OF12GMFL,9,of12.csv,OF12GMFL:of12.csv,OF12GMFL:of12.csv:9


In [13]:
df_columns_count = df_columns.groupby(['column']).agg(count=('column', 'count')).sort_values(by=['count'], ascending=False)
df_columns_count.head(20)

,count
column,
CENTER,10
REC_CMP,10
CRT_DATE,10
CMP_DATE,10
LHFOLNUM,10
BIRTHDT,1
NF4OTHSP,1
NF5AKLLT,1
NF5ABVOC,1


### 03.3 check LHFOLNUM

In [14]:
is_LM3ID = df_columns['column'] == 'LHFOLNUM'
df_columns[is_LM3ID]

,column,idx,filename,column:filename,column:filename:idx
0,LHFOLNUM,0,of01.csv,LHFOLNUM:of01.csv,LHFOLNUM:of01.csv:0
32,LHFOLNUM,1,of03.csv,LHFOLNUM:of03.csv,LHFOLNUM:of03.csv:1
106,LHFOLNUM,2,of04.csv,LHFOLNUM:of04.csv,LHFOLNUM:of04.csv:2
178,LHFOLNUM,3,of04a.csv,LHFOLNUM:of04a.csv,LHFOLNUM:of04a.csv:3
188,LHFOLNUM,4,of04ar.csv,LHFOLNUM:of04ar.csv,LHFOLNUM:of04ar.csv:4
199,LHFOLNUM,5,of05.csv,LHFOLNUM:of05.csv,LHFOLNUM:of05.csv:5
296,LHFOLNUM,6,of09a.csv,LHFOLNUM:of09a.csv,LHFOLNUM:of09a.csv:6
342,LHFOLNUM,7,of10.csv,LHFOLNUM:of10.csv,LHFOLNUM:of10.csv:7
356,LHFOLNUM,8,of11.csv,LHFOLNUM:of11.csv,LHFOLNUM:of11.csv:8
373,LHFOLNUM,9,of12.csv,LHFOLNUM:of12.csv,LHFOLNUM:of12.csv:9


##### column not in dict

In [15]:
is_not_in_dict = df_columns['column'].isin(dict_columns) == False
df_columns_not_in_dict = df_columns[is_not_in_dict]
df_columns_not_in_dict

,column,idx,filename,column:filename,column:filename:idx
30,CMP_DATE,0,of01.csv,CMP_DATE:of01.csv,CMP_DATE:of01.csv:0
31,CRT_DATE,0,of01.csv,CRT_DATE:of01.csv,CRT_DATE:of01.csv:0
3,FIRST,0,of01.csv,FIRST:of01.csv,FIRST:of01.csv:0
2,LAST,0,of01.csv,LAST:of01.csv,LAST:of01.csv:0
5,PROTID02,0,of01.csv,PROTID02:of01.csv,PROTID02:of01.csv:0
...,...,...,...,...,...
435,CMP_DATE,9,of12.csv,CMP_DATE:of12.csv,CMP_DATE:of12.csv:9
436,CRT_DATE,9,of12.csv,CRT_DATE:of12.csv,CRT_DATE:of12.csv:9
402,LFINITA,9,of12.csv,LFINITA:of12.csv,LFINITA:of12.csv:9
406,NF12INIT,9,of12.csv,NF12INIT:of12.csv,NF12INIT:of12.csv:9


In [16]:
columns_not_in_dict = list(df_columns_not_in_dict['column:filename:idx'].unique())
# columns_not_in_dict.sort()
columns_not_in_dict

['CMP_DATE:of01.csv:0',
 'CRT_DATE:of01.csv:0',
 'FIRST:of01.csv:0',
 'LAST:of01.csv:0',
 'PROTID02:of01.csv:0',
 'PROTID03:of01.csv:0',
 'PROTID04:of01.csv:0',
 'PROTID05:of01.csv:0',
 'PROTID06:of01.csv:0',
 'PROTID07:of01.csv:0',
 'PROTID08:of01.csv:0',
 'PROTID09:of01.csv:0',
 'PROTID10:of01.csv:0',
 'PROTID11:of01.csv:0',
 'PROTID12:of01.csv:0',
 'PROTID13:of01.csv:0',
 'PROTID14:of01.csv:0',
 'PROTID15:of01.csv:0',
 'PROTID16:of01.csv:0',
 'PROTID17:of01.csv:0',
 'PROTID18:of01.csv:0',
 'PROTID19:of01.csv:0',
 'PROTID20:of01.csv:0',
 'REC_CMP:of01.csv:0',
 'CFINITS:of03.csv:1',
 'CFPRRESP:of03.csv:1',
 'CMP_DATE:of03.csv:1',
 'CRT_DATE:of03.csv:1',
 'REC_CMP:of03.csv:1',
 'CMP_DATE:of04.csv:2',
 'CRT_DATE:of04.csv:2',
 'DFCOMPDT:of04.csv:2',
 'DFINITS:of04.csv:2',
 'DFPLACE:of04.csv:2',
 'NF4LOCSP:of04.csv:2',
 'REC_CMP:of04.csv:2',
 'CMP_DATE:of04a.csv:3',
 'CRT_DATE:of04a.csv:3',
 'DFINITSA:of04a.csv:3',
 'REC_CMP:of04a.csv:3',
 'CMP_DATE:of04ar.csv:4',
 'CRT_DATE:of04ar.csv:4'

#### dict not in column

In [17]:
is_dict_not_in_columns = (df_dict['OC'].isin(df_columns['column']) == False) & (df_dict['OC'].isnull() == False)
df_dict_not_in_columns = df_dict[is_dict_not_in_columns]
df_dict_not_in_columns

,Category,Subcategory,Standardized_VariableNames_Dictionary,type,Variable_Description,#studies w/ this var,redcap,comment,LH,OC


## 04. show column values

In [18]:
# XXX no show due to data privacy concern.

'''
for idx, each_df in enumerate(dfs):
    for idx2, column in enumerate(each_df.columns):
        print(f'({idx}/{len(dfs)}/{filenames[idx]}) ({idx2}/{len(each_df.columns)} column: {column} value: {list(each_df[column].unique())}')
'''

"\nfor idx, each_df in enumerate(dfs):\n    for idx2, column in enumerate(each_df.columns):\n        print(f'({idx}/{len(dfs)}/{filenames[idx]}) ({idx2}/{len(each_df.columns)} column: {column} value: {list(each_df[column].unique())}')\n"